# Ensemble from submission CSV only

### This is a CPU only kernel

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import cv2
import os
from tqdm import tqdm
from pathlib import Path

Configs

In [3]:
MIN_SIZE = 10000
HOME = Path(os.environ["HOME"])
INPUT_DIR = Path("/kaggle/input")
SUBS = INPUT_DIR/"subcsv"

Check sub files

In [4]:
!ls -l {SUBS}

total 378064
-rw-r--r-- 1 root root 21579597 Nov 18 15:25 1110_154142_submission.csv
-rw-r--r-- 1 root root 21251497 Nov 18 15:25 1114_090732_submission.csv
-rw-r--r-- 1 root root 21085658 Nov 18 15:25 b5_fold_optimize_trian0.5_1114_150440_submission.csv
-rw-r--r-- 1 root root 22902693 Nov 18 15:25 b5_pla_0.72_1573548650_submission.csv
-rw-r--r-- 1 root root 23107546 Nov 18 15:25 b5_r2_1573525424_submission.csv
-rw-r--r-- 1 root root 23211737 Nov 18 15:25 b6_fold_1114_121001_submission.csv
-rw-r--r-- 1 root root 22315669 Nov 18 15:25 b6_ranger_submission.csv
-rw-r--r-- 1 root root 20280347 Nov 18 15:25 b6_ver2_submission.csv
-rw-r--r-- 1 root root 21339234 Nov 18 15:25 convex_1110_154142_submission.csv
-rw-r--r-- 1 root root 20998002 Nov 18 15:25 convex_1111_161204_submission.csv
-rw-r--r-- 1 root root 20853638 Nov 18 15:25 convex_b5_fold_optimize_trian0.5_1114_150440_submission.csv
-rw-r--r-- 1 root root 21170814 Nov 18 15:25 convex_b6_fold_train0.5_optima_1116_080218_submission.csv
-

Set submission csv pool

In [5]:
# This has to be an odd number
sub_fnames = list([
#     ("resnext_miracle_raw.csv", 0.3333333),# resnext 101 1st miracle 0.6596
#     ("1110_154142_submission.csv",1/7), # enesemble 
#     ("b5_pla_0.72_1573548650_submission.csv",0.33333333)
#     ("b5_r2_1573525424_submission.csv",0.3333333)finding
#     ("dpn_131_0.73_1573574797_submission.csv",0.333333) #0.6443
#     ("b5_fold_optimize_trian0.5_1114_150440_submission.csv",1/7), # 0.6587
#     ("b6_ver2_submission.csv",1/7), #0.6527
#     ("b6_ranger_submission.csv",1/7), #0.6556
#     ("1114_090732_submission.csv",1/7), # resnext 101 5 folds 0.6569
#     ("b6_fold_1114_121001_submission.csv",1/7), # b6 6 folds  0.6558
#     ("convex_b5_fold_optimize_trian0.5_1114_150440_submission.csv", 0.3333333), # convex b5 5 folds
#     ("convex_b6_fold_train0.5_optima_1116_080218_submission.csv", 0.3333333), # convex b6 5 bolds
    ("emp_convex_1110_154142_submission.csv",0.3),
    ("emp_640x960_b6_single_1118_101912_submission.csv",0.1),
    ("emp_se_resnext50_640x960_5folds_1118_082125_submission.csv",0.3),
    ("emp_convex_384x567_1117_211214_submission.csv",0.3),
])
sub_paths = list(SUBS/p[0] for p in sub_fnames)
weighted = list(p[1] for p in sub_fnames)

In [6]:
print(weighted)

[0.3, 0.1, 0.3, 0.3]


In [7]:
def bringOrder(df):
    return df.sort_values(by="Image_Label", ascending=True).reset_index().drop("index",axis=1)

In [8]:
sub_dfs = list(bringOrder(pd.read_csv(p)) for p in sub_paths)
sample_df = sub_dfs[0]

### Helper Functions

In [9]:

def rle_decode(mask_rle: str = '', shape = (350,525 )):
    '''
    Decode rle encoded mask.
    
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(shape, order='F')

def post_process(probability, threshold=0.5, min_size = MIN_SIZE):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = cv2.threshold(np.float32(probability), threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

def mask2rle(img):
    '''
    Convert mask to rle.
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

### Stacking with csv
* CPU only
* Not running any model at this point
* CSV files has to be even number
* The ensembled probability will go through the min size again

In [10]:

encoded_pixels = []
for row_idx in tqdm(range(len(sample_df)//4)):
    row_list = []
    for cls in range(4):
        img_pred = []
        for i in range(len(sub_dfs)):
            df = sub_dfs[i]
            
            rl = df.loc[row_idx*4+cls]["EncodedPixels"]
            if type(rl)==str:
                img_pred.append(rle_decode(rl)*weighted[i])
            else:
                img_pred.append(rle_decode(''))
#         if row_idx == 0:
#             print(img_pred)
        # averaging over submissions
        row_prob = np.sum(img_pred,axis=0)
        row_pred, row_pred_num = post_process(row_prob)
        if row_pred_num == 0:
            encoded_pixels.append('')
        else:
            encoded_pixels.append(mask2rle(row_pred))



100%|██████████| 3698/3698 [02:00<00:00, 30.73it/s]


### Saving to submission CSV

In [11]:
from datetime import datetime

In [12]:
sample_df['EncodedPixels'] = encoded_pixels
sample_df.to_csv('csvs_%s_sub.csv'%(datetime.now().strftime("%m%d_%H%M%S")), columns=['Image_Label', 'EncodedPixels'], index=False)